In [1]:
inputs = read.csv("https://raw.githubusercontent.com/corydonbaylor/supply-chain/refs/heads/main/data/inputs.csv")
providers = read.csv("https://raw.githubusercontent.com/corydonbaylor/supply-chain/refs/heads/main/data/providers.csv")
provision = read.csv("https://raw.githubusercontent.com/corydonbaylor/supply-chain/refs/heads/main/data/provision.csv")
seq = read.csv("https://raw.githubusercontent.com/corydonbaylor/supply-chain/refs/heads/main/data/sequence.csv")

In [2]:
install.packages("tidyverse")
library(tidyverse)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Cleaning Provisions

Two goals here:


1.   appx the market share
2.   just have organizations


### Market Share
The end goal of this is to have a reasonable guess of the market share. We will use this as weights.

There is two issues:


1.   Many inputs have nothing in the market share
2.   Some are listed as being neglible

We will assume equal market shares among providers UNLESS they are marked as being neglible in the market. In which case, we will say that they have 1% share and that the others split the market equally.

For example:
Discrete GPU has AMD, Nvidia, and Jingjia Micro (which is neglible), so their weights will be:


*   Nvidia .50
*   AMD .50
*   Jingjia Micro .01




In [3]:
# first we need to convert to decimal
provision$share_provided <- as.numeric(sub("%", "", provision$share_provided)) / 100

Next, we are going to make an educated guess at a market share. If its neglible we will put its share at 1% and take it out of its group

In [4]:
provision = provision%>%mutate(group = ifelse(negligible_market_share == "negligible", "small", provided_id))

Next we are going to equally divide the market share amongst the vendors based on if they provide that input

In [5]:
provision = provision%>%
  group_by(group)%>%
  mutate(share = ifelse(
    group == "small", .01,
    ifelse(is.na(share_provided), 1/n(), share_provided)
  )
  )%>%ungroup

Next, I would like to **just** consider organizations rather than countries. We first need to get a list of organizations:

In [6]:
orgs = providers%>%filter(provider_type == "organization")

Then filter them out of the provision table

In [7]:
org_prov = provision %>% filter(provider_id %in% orgs$provider_id)

In [8]:
org_prov%>%
  filter(provider_id == "P34")

provider_name,provider_id,provided_name,provided_id,share_provided,negligible_market_share,group,share
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
TSMC,P34,Fabrication,S2,NA,,S2,0.05555556
TSMC,P34,"Assembly, testing, and packaging (ATP)",S3,NA,,S3,0.04761905
TSMC,P34,Advanced photomasks,N33,NA,,N33,0.08333333


In [9]:
head(org_prov)

provider_name,provider_id,provided_name,provided_id,share_provided,negligible_market_share,group,share
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Intel,P9,Logic chip design: Advanced CPUs,N1,NA,,N1,0.50
AMD,P10,Logic chip design: Advanced CPUs,N1,NA,,N1,0.50
Nvidia,P11,Logic chip design: Discrete GPUs,N2,NA,,N2,0.50
Jingjia Micro,P12,Logic chip design: Discrete GPUs,N2,NA,negligible,small,0.01
Loongson,P13,Logic chip design: Advanced CPUs,N1,NA,negligible,small,0.01
Zhaoxin,P14,Logic chip design: Advanced CPUs,N1,NA,negligible,small,0.01


## Cleaning up the Sequence Table

We want to have the following graph model:

`(provider) -[outputs]-> (input) -[input]-> (provider)`

Lets walk through an example:

`(N8: Crystal Growing Furnaces)` is an input to `[Shin-Etsu, SUMCO, or GlobalWafers]` who makes `(wafers)`

We already have the outputs relationship captured in the provision table. But we need the inputs captured.


### Removing type of in the sequence

N20-22 are types of N19. Ideally, this subset of inputs wouldnt exist.

In [10]:
head(seq)

,input_name,input_id,goes_into_name,goes_into_id,is_type_of_name,is_type_of_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Crystal growing furnaces,N8,Wafer,N26,,
2,Crystal machining tools,N9,Wafer,N26,,
3,EUV scanners,N20,,,Advanced photolithography equipment,N19
4,ArF scanners,N21,,,Advanced photolithography equipment,N19
5,ArF immersion scanners,N22,,,Advanced photolithography equipment,N19
6,Advanced photolithography equipment,N19,Photolithography,N25,,


In [24]:
types2

input_name,input_id,goes_into_name,is_type_of_name,is_type_of_id,goes_into_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
EUV scanners,N20,,Advanced photolithography equipment,N19,N25
ArF scanners,N21,,Advanced photolithography equipment,N19,N25
ArF immersion scanners,N22,,Advanced photolithography equipment,N19,N25
Electron-beam lithography tools,N29,,Photomask (maskless) lithography equipment,N28,N33
Laser lithography tools,N30,,Photomask (maskless) lithography equipment,N28,N33
Plasma CVD tools,N38,,Chemical vapor deposition tools,N47,N35
Low-pressure CVD tools,N39,,Chemical vapor deposition tools,N47,N35
High-temperature CVD tools,N40,,Chemical vapor deposition tools,N47,N35
Atomic layer deposition tools,N41,,Chemical vapor deposition tools,N47,N35


In [11]:


# this represents inputs that directly go into another input
seq_x = seq %>% filter(grepl("N|S", goes_into_id))

# this represents inputs that are a type of another input
types = seq %>% filter(!grepl("N|S", goes_into_id))

# we are going to make a crosswalk of those inputs
seq_x_slim = seq%>%select(input_id, goes_into_id)

# we then join types to seq df to find what the supergroup goes into
# for example, N20-22 are a type of N19 and N19 goes into N25
# therefore, N20-22 goes into N25
types2 = types%>%left_join(seq_x_slim, by = c("is_type_of_id" = "input_id"))%>%
  select(-goes_into_id.x)%>%
  rename(goes_into_id= goes_into_id.y)

head(types2)

,input_name,input_id,goes_into_name,is_type_of_name,is_type_of_id,goes_into_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,EUV scanners,N20,,Advanced photolithography equipment,N19,N25
2,ArF scanners,N21,,Advanced photolithography equipment,N19,N25
3,ArF immersion scanners,N22,,Advanced photolithography equipment,N19,N25
4,Electron-beam lithography tools,N29,,Photomask (maskless) lithography equipment,N28,N33
5,Laser lithography tools,N30,,Photomask (maskless) lithography equipment,N28,N33
6,Plasma CVD tools,N38,,Chemical vapor deposition tools,N47,


In [12]:
# only include types and not super groups like "Dry etching and cleaning tools"
# unforunately have to hard code a few things:
types2 = types2%>% mutate(goes_into_id = ifelse(is_type_of_id == "N47", "N35", goes_into_id))
types2 = types2%>% mutate(goes_into_id = ifelse(is_type_of_id == "N48", "N55", goes_into_id))

head(types2)

,input_name,input_id,goes_into_name,is_type_of_name,is_type_of_id,goes_into_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,EUV scanners,N20,,Advanced photolithography equipment,N19,N25
2,ArF scanners,N21,,Advanced photolithography equipment,N19,N25
3,ArF immersion scanners,N22,,Advanced photolithography equipment,N19,N25
4,Electron-beam lithography tools,N29,,Photomask (maskless) lithography equipment,N28,N33
5,Laser lithography tools,N30,,Photomask (maskless) lithography equipment,N28,N33
6,Plasma CVD tools,N38,,Chemical vapor deposition tools,N47,N35


In [35]:
# now we need to combine types2 and seq_x
sequence = seq_x%>%bind_rows(types2)
# we still have the same number of rows as when we started so that is good
var = "N43"
sequence%>%
  filter(input_id == var | goes_into_id == var)

input_name,input_id,goes_into_name,goes_into_id,is_type_of_name,is_type_of_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Rapid thermal processing tools,N43,,N35,Deposition tools,N36


### Finding Inputs (to a Provider)
Next we need to find out what inputs each provider needs before it can make an output.

Let's walk through an example.
- For N26 (wafers), we require N8 and N9.
- N8 has a list of suppliers in the org_prov that we could use
- that list is all the potential suppliers for N8

In [14]:
# next we need to know what suppliers each input relies on
provision_slim = org_prov%>%select(provider_id, provided_id)
head(provision_slim)

provider_id,provided_id
<chr>,<chr>
P9,N1
P10,N1
P11,N2
P12,N2
P13,N1
P14,N1


In [15]:
provision_slim%>%filter(provided_id == "N33")

provider_id,provided_id
<chr>,<chr>
P238,N33
P239,N33
P240,N33
P241,N33
P242,N33
P9,N33
P35,N33
P34,N33
P36,N33


In [16]:
sequence%>%
  filter(goes_into_id == "N97")

input_name,input_id,goes_into_name,goes_into_id,is_type_of_name,is_type_of_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [17]:
a = provision_slim%>%
  filter(grepl("S", provided_id))%>%
  left_join(inputs, by = c("provided_id" = "stage_id"))%>%
  select(provider_id, input_id)%>%
  rename(provided_id = input_id)

b = provision_slim%>%
  filter(grepl("N", provided_id))

final_provision = rbind(a,b)

Warning message in left_join(., inputs, by = c(provided_id = "stage_id")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 15 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [18]:
inputs_to_providers = final_provision %>%
  left_join(sequence, by = c("provided_id" = "goes_into_id"))%>%
  select(provider_id, provided_id, input_id)

nrow(inputs_to_providers)

inputs_to_providers%>%
  filter(provided_id == "N78")
#write.csv(inputs_to_providers, "inputs_to_providers.csv", row.names=F)


Warning message in left_join(., sequence, by = c(provided_id = "goes_into_id")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 3 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


[1] 1427

provider_id,provided_id,input_id
<chr>,<chr>,<chr>
P49,N78,N69
P49,N78,N80
P49,N78,N81
P49,N78,N82
P49,N78,N83
P50,N78,N69
P50,N78,N80
P50,N78,N81
P50,N78,N82


In [19]:
write.csv(inputs_to_providers, "inputs_to_providers.csv", row.names=F)


In [21]:
write.csv(final_provision, "outputs.csv", row.names=F)

In [138]:
write.csv(sequence, "sequence_final.csv", row.names=F)

In [139]:
write.csv(orgs, "organizatons.csv", row.names=F)